In [1]:
from typing import Dict, List, Tuple
import matplotlib.pyplot as plt 
import networkx as nx 
import numpy as np
import pandas as pd 
import matplotlib

from scipy import stats

In [2]:
pneumoniae = pd.read_excel("../data/12859_2019_2897_MOESM2_ESM.xlsx", 
                       sheet_name=3, index_col=0)

# remove whitespace from pneumoniae
pneumoniae.index = pneumoniae.index.str.strip()
pneumoniae_nx = nx.from_numpy_matrix(np.array(pneumoniae))
ess_rxn_pneu = pd.read_csv("essential_rxns_pneu.txt",
                          header=None)
# restrict essential reactions to those found in adjacency matrix
ess_rxn_pneu = pd.DataFrame(pneumoniae[pneumoniae.index.isin(ess_rxn_pneu[0])].index)

### Functions for plotting

In [3]:
def bridging_centrality(graph: nx.Graph) -> Dict:
    """
    Compute (modified) bridging centrality as defined in methods 
    section.
    
    :param graph: networkx (directed) graph 
    :return: Dict mapping nodes to briding centrality 
    """
    degrees_all = dict(nx.degree(graph))
    degree_in = dict(nx.DiGraph(graph).in_degree())
    degree_out = dict(nx.DiGraph(graph).out_degree())
    
    between_centr = nx.betweenness_centrality(graph)
    
    bc = {} # map node to bridging centrality 
    for rxn in degrees_all:
        if degree_in[rxn] == 0 or degree_out[rxn] == 0:
            bc[rxn] = 0
        else:
            numerator = 1/degrees_all[rxn]
            denominator = np.sum([1/degrees_all[x] for x in 
                                  graph.neighbors(rxn)])
            bc[rxn] = (numerator / denominator)*between_centr[rxn]
    return bc 

In [4]:
def prepare_graphing_data(metric: List, 
        essential: pd.DataFrame) -> List:
    """
    Count share of essential/non-essential reactions in each bin, percent 
    essential within a bin.
    
    :param hist_bins: List of (rounded) graph statistic values. 
    :param essential: DataFrame of essential reactions.
    :return: Essential reactions, nonessential reactions, % essential 
             within each bin 
    """
    # get essential reaction indices 
    ess_idx = np.where(pneumoniae.index.isin(essential[0]))[0].tolist()
    
    # count *all* reactions in each bin
    # drop any inf's if there are any (an issue when we take log)
    all_rxn_per_bin = np.unique([x for x in metric if x != -np.inf], return_counts=True)
    
    # count *essential* reactions in each bin (including bins with 0 ess!)
    ess_only = np.array(metric)[ess_idx]
    ess_rxn_per_bin = [(ess_only == x).sum() 
                       for x in all_rxn_per_bin[0]]

    # get percent essential per bin 
    perc_ess_per_bin = [x/y for x,y in zip(ess_rxn_per_bin,
                                           all_rxn_per_bin[1])]
    # get count of non-essential per bin
    non_ess_per_bin = [x-y for x,y in zip(all_rxn_per_bin[1], 
                                          ess_rxn_per_bin)]

    return ess_rxn_per_bin, non_ess_per_bin, perc_ess_per_bin

In [5]:
def construct_plot(ess: List, non_ess: List, perc_ess: List,
                   x_axis: List, x_title: str, out_path: str) -> None:
    """
    Generate the combined histogram and scatterplot from the corresponding data. 
    
    :param ess: List of counts of essential reactions per bin. 
    :param non_ess: List of counts of non-essential reactions per bin. 
    :param perc_ess: List of percent of essential reactions per bin. 
    :param x_axis: List of x axis tick *labels* that match the paper directly. 
    :param x_title: x-axis title corresponding to graph metric. 
    :param out_path: Where to save figure. 
    """
    spacing = [x for x in range(len(x_axis))]
    fig, ax1 = plt.subplots()
    ax1.bar(spacing, ess, color="black")
    ax1.bar(spacing, non_ess, color="gray", bottom=ess)
    ax1.set_ylabel("Number of Reactions")
    plt.xticks(spacing, x_axis)
    plt.xlabel(x_title)

    ax2 = ax1.twinx()
    ax2.set_ylabel("% Essential")
    ax2.scatter(spacing, perc_ess, color="black")
    # line of best fit 
    m, b = np.polyfit(spacing, perc_ess, 1) # still plotting line
    ax2.plot(spacing, [(m*x) + b for x in spacing], color="black")

    # correlation
    lr = stats.linregress(spacing, perc_ess)
    plt.plot([], [], ' ', label = f"r={round(np.corrcoef(perc_ess, spacing)[0,1],2)}, p-value = {round(lr.pvalue,4)}, r-squared={lr.rvalue**2:.6}")
    plt.legend(frameon=False, prop={'size': 12}, bbox_to_anchor=(1.05, 1.0), loc='upper right')
    
    plt.savefig(out_path)
    plt.clf()

### Panel a: log(bridging centrality)

In [6]:
bridge_centr = bridging_centrality(pneumoniae_nx)
log10_bridge_centr = np.log10(list(bridge_centr.values()))
ess, non_ess, perc_ess = prepare_graphing_data([round(x*2)/2 if x != -np.inf else -np.inf for x in log10_bridge_centr], ess_rxn_pneu)
construct_plot(ess[:-1], non_ess[:-1], perc_ess[:-1], [x/2 for x in range(len(ess)-1)], r'$log_{10}$(Bridging Centrality)', "panela_pval")

<ipython-input-6-71c17dd2bb9f>:2: RuntimeWarning: divide by zero encountered in log10
  log10_bridge_centr = np.log10(list(bridge_centr.values()))


<Figure size 432x288 with 0 Axes>

### Panel b: log(betweenness centrality)

In [7]:
between_centr = list(nx.betweenness_centrality(pneumoniae_nx).values())
log10_between_centr = np.log10(between_centr)

## shift to the right by the minimum positive log10 value 
min_pos_log10bc = np.min([x for x in log10_between_centr if x != -np.inf])
log10_between_centr = [x + np.abs(min_pos_log10bc) if x != -np.inf else -np.inf for x in log10_between_centr]
##

ess, non_ess, perc_ess = prepare_graphing_data([round(x*2)/2 if x != -np.inf else -np.inf for x in log10_between_centr], ess_rxn_pneu)
construct_plot(ess, non_ess, perc_ess, [x/2 for x in range(len(ess))], r'$log_{10}$(Betweenness Centrality)', "panelb_pval")



<ipython-input-7-a34e3597f396>:2: RuntimeWarning: divide by zero encountered in log10
  log10_between_centr = np.log10(between_centr)


<Figure size 432x288 with 0 Axes>

### Panel c: clustering coefficient

In [8]:
clust_coef = list(nx.clustering(pneumoniae_nx).values())
ess, non_ess, perc_ess = prepare_graphing_data([max(0.1, round(x,1)) for x in clust_coef], ess_rxn_pneu)
construct_plot(ess, non_ess, perc_ess, [(x+1)/10 for x in range(len(ess))], "Clustering Coefficient", "panelc_pval")

<Figure size 432x288 with 0 Axes>

### Panel d: log(degree)

In [9]:
degree = list(dict(nx.degree(pneumoniae_nx)).values())
log10_degree = np.log10(degree)
ess, non_ess, perc_ess = prepare_graphing_data([round(x*4)/4 for x in log10_degree], ess_rxn_pneu)
construct_plot(ess, non_ess, perc_ess, [x/4 for x in range(len(ess))], r'$log_{10}$(Degree)', "paneld_pval")

<Figure size 432x288 with 0 Axes>

## Statistical Analysis
1. Plot each of the histograms for centrality metrics for K.Pneumoniae
    - You can determine statistical significance of correlation r-values where the null hypothesis is that the slope for each metric is zero, i.e. no correlation.
2. **Conduct a correlation analysis**: Now that we have each of the p-values and r values for K.Pneumoniae, we proceed to determine whether the correlation coefficients between the histograms of K.Pneumoniae vs Ecoli are statistically significant.
    - Can we obtain this via a z-test (two-tailed, since not equal to the r values claimed in the histogram network metrics of Ecoli)?
        - Convert each r value in to a z score (this would allow for the z-scores to be compared and analyzed for statistical significance by determining the observed statistic).
        - Of course one would have to to establish a level of significance or alpha level to properly assess significance.
        - Since correlation coefficients are obtained for both organisms, we take note of the sample sizes (259 reactions in ecoli, and 2262 in pneumoniae) --> *is the sample size the number of essential reactions or all reactio*
        - Once sample sizes are obtained, calculate z observed.
        - Check to see whether the observed value is greater than +/- the critical value, which can be obtained from a z score table. 
        - Calculate p-value based on the obtained value in z table and multiplying it by 2 (since two tailed test).
        

### Regression P-Values for K.Pneumoniae:
- $H_0: r=0 $

- $alpha = 0.05$

- $H_{alt}: r != 0$ 

**Note**: the p-values for regression coefficient of K.Pneumoniae were obtained under the null hypothesis is that the slope is zero, using Wald Test with t-distribution of the test statistic. Also p-values will not be displayed in the plot but as subtext in the graphic to keep everything concise. Below I compile all obtained p-values for safekeeping.

In [10]:
# correlation p_values for K.Pneumoniae 
p_valuesKP = {"Bridging Centrality": 0.159, "Betweenness Centrality":0.0047,
       "Clustering Coefficient":0.0383, "Degree":0.0687}

# correlation r_values for K.Pneumoniae
r_valuesKP = {"Bridging Centrality": 0.48, "Betweenness Centrality":0.73,
       "Clustering Coefficient":-0.66, "Degree":0.0687}

# correlation r_values for Ecoli (authors claim), and my null hypothesis.
r_valuesEC = {"Bridging Centrality": 0.87, "Betweenness Centrality":0.82,
        "Clustering Coefficient":-0.86, "Degree":0.21}

### Z-test
With a two-tailed z-test we can establish statistical significance among the different r coefficients between *K.Pneumoniae* and *Ecoli*.

$H_0: r_{kpneumoniae} = r_{ecoli}$

$H_{alt}: r_{kpneumoniae} \neq r_{ecoli}$

$alpha = 0.05$

In [11]:
# sample sizes (outputs obtained from FBA python notebook)
ecoli_N = 2583
kpneu_N = 2262

1. Convert each $r_{kpneumoniae}$ value to a z-score, then we can compare z-scores for statistical significance by determining the observed z, using the following:

$$z_1 =\frac{1}{2}ln(\frac{1 + r_{kpne}}{1 - r_{kpneu}})$$

$$z_2 = \frac{1}{2}ln(\frac{1 + r_{ecoli}}{1 - r_{ecoli}}) $$

$$z_{observed} = \frac{z_1 - z_2}{\frac{1}{\sqrt{N_{datapts}-3}}}$$


In [12]:
# get z_1's and z_2's for either k pneumoniae or ecoli respectively, ignore negative signs 
z_scoresKP = [0.5*np.log((1 + r)/(1 - r)) for r in r_valuesKP.values()]
z_scoresEC = [0.5*np.log((1 + r)/(1 - r)) for r in r_valuesEC.values()]
std_n1n2 = 1/np.sqrt((10-3))

z_observed = [(z_scoresKP[z] - z_scoresEC[z])/ std_n1n2 for z in range(len(z_scoresKP))]

In [13]:
z_scoresEC, z_scoresKP, z_observed

([1.333079629696525,
  1.156817464590315,
  -1.2933446720489712,
  0.21317134656485978],
 [0.5229842775913438,
  0.9287273642467249,
  -0.792813631870191,
  0.06880838800161941],
 [-2.143310839919615,
  -0.6034696820249079,
  1.324280655781531,
  -0.38194848688785643])

In [14]:
indexes = ["Ecoli r-value", "K.Pneumoniae r-value", "Ecoli z-value", "K.Pneumoniae z-value", "z-score"]
cols_ = r_valuesKP.keys()

#find p-value for two-tailed test



df_complete = pd.DataFrame(index = indexes, columns = cols_)
df_complete.loc['Ecoli r-value'] = [i for i in r_valuesEC.values()]
df_complete.loc['K.Pneumoniae r-value'] = [j for j in r_valuesKP.values()]
df_complete.loc["Ecoli z-value"] = [round(k,2) for k in z_scoresEC]
df_complete.loc["K.Pneumoniae z-value"] = [round(h,2) for h in z_scoresKP]
df_complete.loc["z-score"] = [round(i,2) for i in z_observed]
df_complete.loc["p-vals"] = [stats.norm.sf(abs(z))*2 for z in df_complete.loc["z-score"]]

In [15]:
df_complete

,Bridging Centrality,Betweenness Centrality,Clustering Coefficient,Degree
Ecoli r-value,0.87,0.82,-0.86,0.21
K.Pneumoniae r-value,0.48,0.73,-0.66,0.0687
Ecoli z-value,1.33,1.16,-1.29,0.21
K.Pneumoniae z-value,0.52,0.93,-0.79,0.07
z-score,-2.14,-0.6,1.32,-0.38
p-vals,0.032355,0.548506,0.186835,0.703945
